데이터 전처리 과정
데이터 전처리는 머신러닝 모델의 성능에 중요한 영향을 미칩니다. 여기서는 결측값 처리, 불필요한 열 제거, 범주형 변수 인코딩 등의 작업을 수행합니다.

1. 데이터 로드 및 전처리

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA

# 데이터 파일 경로를 지정합니다.
csv_path = os.getenv("HOME") + "/aiffel/pokemon_eda/data/Pokemon.csv"

# 데이터를 불러옵니다.
df = pd.read_csv(csv_path)

# 데이터의 첫 몇 행을 출력하여 확인합니다.
df.head()

# 결측값이 있는지 확인합니다.
print(df.isnull().sum())

# 숫자형 열의 결측값을 해당 열의 평균값으로 채웁니다.
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df.mean())

# 분석에 필요없는 열을 제거합니다. 예를 들어, 'Name' 열은 포켓몬의 이름이므로 분석에 큰 영향을 주지 않습니다.
df.drop(['Name', 'Type 2'], axis=1, inplace=True)

# 범주형 변수를 숫자로 변환합니다. 여기서는 'Type 1'이 범주형 변수입니다.
df = pd.get_dummies(df, columns=['Type 1'])

# 'Legendary' 열이 우리가 예측하려는 타겟 변수입니다.
X = df.drop('Legendary', axis=1)
y = df['Legendary']

# 데이터를 학습용과 테스트용으로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 학습 데이터를 표준화합니다. (평균을 0, 표준편차를 1로 맞춥니다.)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64


/tmp/ipykernel_409/3804938006.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[numeric_cols] = df[numeric_cols].fillna(df.mean())


2. 베이스라인 모델 학습

In [7]:
# 랜덤 포레스트 분류기를 학습시킵니다.
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터에 대해 예측을 수행합니다.
y_pred = model.predict(X_test)

# 정확도를 계산합니다.
accuracy = accuracy_score(y_test, y_pred)
print(f'베이스라인 모델의 정확도: {accuracy:.2f}')


베이스라인 모델의 정확도: 0.97


3. 성능 향상 노력
3.1. 특성 선택 (Feature Selection)

In [8]:
# 특성 중요도를 확인합니다.
feature_importances = model.feature_importances_
features = X.columns

# 중요도가 높은 특성을 선택합니다.
important_features = [features[i] for i in np.argsort(feature_importances)[-10:]]
print(f'선택된 중요 특성: {important_features}')

X_train_selected = X_train[:, np.argsort(feature_importances)[-10:]]
X_test_selected = X_test[:, np.argsort(feature_importances)[-10:]]

# 선택된 특성으로 다시 모델을 학습시킵니다.
model_selected = RandomForestClassifier(random_state=42)
model_selected.fit(X_train_selected, y_train)

# 테스트 데이터에 대해 예측을 수행합니다.
y_pred_selected = model_selected.predict(X_test_selected)

# 정확도를 계산합니다.
accuracy_selected = accuracy_score(y_test, y_pred_selected)
print(f'특성 선택 후 모델의 정확도: {accuracy_selected:.2f}')


선택된 중요 특성: ['Type 1_Psychic', 'Generation', 'Defense', 'Sp. Def', 'Attack', 'HP', 'Speed', '#', 'Sp. Atk', 'Total']
특성 선택 후 모델의 정확도: 0.99


3.2. 모델 튜닝 (Model Tuning)

In [9]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 그리드 서치 적용
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적의 모델을 선택합니다.
best_model = grid_search.best_estimator_

# 테스트 데이터에 대해 예측을 수행합니다.
y_pred_best = best_model.predict(X_test)

# 정확도를 계산합니다.
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'모델 튜닝 후 정확도: {accuracy_best:.2f}')


Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   1.0s
[CV] END m

3.3. PCA 적용 (Principal Component Analysis)

In [12]:
# PCA를 사용하여 차원을 축소합니다.
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# 축소된 데이터로 모델을 학습시킵니다.
model_pca = RandomForestClassifier(random_state=42)
model_pca.fit(X_train_pca, y_train)

# 테스트 데이터에 대해 예측을 수행합니다.
y_pred_pca = model_pca.predict(X_test_pca)

# 정확도를 계산합니다.
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print(f'PCA 적용 후 모델의 정확도: {accuracy_pca:.2f}')


PCA 적용 후 모델의 정확도: 0.96


4. 독창적인 성능 향상 방법
4.1. 앙상블 학습 (Ensemble Learning)
앙상블 학습은 여러 모델을 결합하여 더 나은 예측 성능을 얻는 방법입니다. 여기서는 여러 모델의 예측 결과를 평균내어 최종 예측을 만드는 간단한 앙상블 방법을 사용해보겠습니다.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 로지스틱 회귀 모델 학습
model_lr = LogisticRegression(random_state=42)
model_lr.fit(X_train, y_train)

# 서포트 벡터 머신 모델 학습
model_svc = SVC(probability=True, random_state=42)
model_svc.fit(X_train, y_train)

# 각 모델의 예측 결과를 얻습니다.
pred_rf = model.predict_proba(X_test)
pred_lr = model_lr.predict_proba(X_test)
pred_svc = model_svc.predict_proba(X_test)

# 예측 결과의 평균을 계산합니다.
pred_ensemble = (pred_rf + pred_lr + pred_svc) / 3

# 평균된 예측 결과에서 가장 높은 확률을 가진 클래스를 선택합니다.
y_pred_ensemble = np.argmax(pred_ensemble, axis=1)

# 정확도를 계산합니다.
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f'앙상블 모델의 정확도: {accuracy_ensemble:.2f}')


앙상블 모델의 정확도: 0.96


4.2. 새로운 특성 생성 (Feature Engineering)
새로운 특성을 생성하여 모델의 성능을 향상시키는 방법입니다. 여기서는 기존의 특성들을 조합하여 새로운 특성을 만들어보겠습니다. 예를 들어, Attack과 Defense의 차이를 새로운 특성으로 추가합니다.

In [14]:
# 새로운 특성을 생성합니다.
df['Attack_Defense_Diff'] = df['Attack'] - df['Defense']
df['Speed_Defense_Ratio'] = df['Speed'] / (df['Defense'] + 1)  # 0으로 나누는 것을 방지하기 위해 +1을 합니다.

# 새롭게 생성된 특성을 포함하여 데이터 준비
X = df.drop('Legendary', axis=1)
y = df['Legendary']

# 데이터를 학습용과 테스트용으로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 학습 데이터를 표준화합니다.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 새로운 특성을 포함한 데이터로 모델을 학습시킵니다.
model_with_new_features = RandomForestClassifier(random_state=42)
model_with_new_features.fit(X_train, y_train)

# 테스트 데이터에 대해 예측을 수행합니다.
y_pred_new_features = model_with_new_features.predict(X_test)

# 정확도를 계산합니다.
accuracy_new_features = accuracy_score(y_test, y_pred_new_features)
print(f'새로운 특성 추가 후 모델의 정확도: {accuracy_new_features:.2f}')


새로운 특성 추가 후 모델의 정확도: 0.98
